In [10]:
import bcrypt
from twisted.internet import protocol, reactor
from twisted.protocols import basic

In [15]:
class ChatServerProtocol(basic.LineOnlyReceiver):
    def __init__(self, factory):
        self.factory = factory
        self.name = None
        self.authenticated = False

    def connectionMade(self):
        self.sendLine(b"Welcome to the Twisted Chat!")
        self.sendLine(b"Please enter your name:")

    def lineReceived(self, line):
        try:
            if not self.authenticated:
                if self.name is None:
                    self.name = line.decode().strip()
                    self.sendLine(b"Please enter your password:")
                elif self.password is None:
                    self.password = line.decode().strip()
                    self.sendLine(b"Please enter your password again:")
                elif self.password_confirm is None:
                    self.password_confirm = line.decode().strip()
                    if self.password == self.password_confirm:
                        if self.register(self.name, self.password):
                            self.authenticated = True
                            self.factory.clients[self] = self.name
                            self.sendLine(f"Registration successful. Welcome, {self.name}!".encode())
                            self.sendLine(b"Type 'quit' to exit.")
                        else:
                            self.sendLine(b"Error occurred during registration. Please try again.")
                    else:
                        self.sendLine(b"Passwords do not match. Please try again.")
            else:
                message = line.decode().strip()
                if message == 'quit':
                    self.transport.loseConnection()
                else:
                    self.factory.broadcast(f"{self.name}: {message}")
        except ValueError as e:
            self.sendLine(f"Error: {e}".encode())
        except Exception as e:
            self.sendLine(b"Error occurred. Please try again.")


    def connectionLost(self, reason):
        if self in self.factory.clients:
            del self.factory.clients[self]
            self.factory.broadcast(f"{self.name} has left the chat.")
        self.cursor.close() 
        self.conn.close()  
        
    def register(self, username, password):
        try:
            if username in self.factory.users:
                self.sendLine(b"Username already taken. Please choose a different username.")
                return False
            else:
                password_bytes = password.encode('utf-8')
                salt = bcrypt.gensalt()
                password_hash = bcrypt.hashpw(password_bytes, salt).decode('utf-8')
                self.cursor.execute("INSERT INTO users (username, password_hash) VALUES (%s, %s);", (username, password_hash))
                self.conn.commit()
                self.factory.users[username] = password_hash
                return True
        except Exception as e:
            self.sendLine(b"Error occurred during registration. Please try again.")
            print(f"Error: {e}")
            return False

    def authenticate(self, username, password):
        try:
            self.cursor.execute("SELECT password_hash FROM users WHERE username = %s;", (username,))
            row = self.cursor.fetchone()
            if row is not None:
                stored_password_hash = row[0]
                password_bytes = password.encode('utf-8')
                stored_password_hash_bytes = stored_password_hash.encode('utf-8')
                return bcrypt.checkpw(password_bytes, stored_password_hash_bytes)
            return False
        except Exception as e:
            self.sendLine(b"Error occurred while authenticating. Please try again.")
            print(f"Error: {e}")
            return False


In [16]:
class ChatServerFactory(protocol.Factory):
    def __init__(self):
        self.clients = {}
        self.users = {}
        self.conn = psycopg2.connect(
            dbname="your_db_name",  # Replace with your database name
            user="your_username",  # Replace with your database username
            password="your_password",  # Replace with your database password
            host="your_host",  # Replace with your database host
            port="your_port"  # Replace with your database port
        )
        self.cursor = self.conn.cursor()

In [18]:
if __name__ == "__main__":
    try:
        factory = ChatServerFactory()
        factory.cursor.execute("SELECT username, password_hash FROM users;")
        rows = factory.cursor.fetchall()
        for row in rows:
            factory.users[row[0]] = row[1]

        reactor.listenTCP(9000, factory)
        print("Chat server started on port 9000.")
        reactor.run()
    except Exception as e:
        print(f"Error: {e}")

Error: Couldn't listen on any:9000: [WinError 10048] Обычно разрешается только одно использование адреса сокета (протокол/сетевой адрес/порт).
